# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [1]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [5]:
def polynomial_sframe(feature, degree):
    poly_sframe=graphlab.SFrame()
    poly_sframe['power_1']=feature
    if degree > 1:
        for power in range(2,degree+1):
            name='power_'+str(power)
            poly_sframe[name]=feature.apply(lambda x: x**power)
    return poly_sframe
           

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] 1455163095 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /usr/local/lib/python2.7/dist-packages/certifi/cacert.pem
1455163095 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to plutodocs@gmail.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1295 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1455163095.log
[INFO] GraphLab Server Version: 1.7.1


In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1455165233 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /usr/local/lib/python2.7/dist-packages/certifi/cacert.pem
1455165233 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to plutodocs@gmail.com and will expire on January 04, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1435 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1455165233.log
[INFO] GraphLab Server Version: 1.7.1


As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living first:

In [3]:
sales = sales.sort('sqft_living')

Let us revisit the 15th-order polynomial model using the 'sqft_living' as the input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. After fitting the model, print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=0.0` in this call.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [6]:
poly15=polynomial_sframe(sales['sqft_living'], 15)
my_features=poly15.column_names()
poly15['price']=sales['price']
model_poly15=graphlab.linear_regression.create(poly15,'price',features=my_features,validation_set=None,l2_penalty=0.0)

model_poly15.get("coefficients")

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.019304     | 2603444.711060     | 245496.612117 |
PROGRESS: WARNING: Matrix is close to being singular or badly scaled. The solution is accurate only up to a tolerance of 1.2818. This typically happens when regularization is not sufficient. Consider increasing regularization.
PROGRESS: +-----------+----------+--------------+-------

name,index,value
(intercept),None,219835.490721
power_1,None,-519.287136851
power_2,None,2.04868141038
power_3,None,-0.00285843174611
power_4,None,2.24528899734e-06
power_5,None,-1.10391042868e-09
power_6,None,3.6259305564e-13
power_7,None,-8.29779737237e-17
power_8,None,1.3658866814e-20
power_9,None,-1.63941755919e-24


***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use the `.random_split` function and make sure you set `seed=1`. 

In [7]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=1)
(set_1, set_2) = semi_split1.random_split(0.5, seed=1)
(set_3, set_4) = semi_split2.random_split(0.5, seed=1)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=0.0` when calling `graphlab.linear_regression.create()`.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [9]:
set1_poly=polynomial_sframe(set_1['sqft_living'],15)
my_features=set1_poly.column_names()
set1_poly['price']=set_1['price']
set1_0p=graphlab.linear_regression.create(set1_poly,'price',features=my_features,validation_set=None,l2_penalty=0.0)
set1_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5492
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005353     | 2481276.175537     | 251487.535599 |
PROGRESS: WARNING: Matrix is close to being singular or badly scaled. The solution is accurate only up to a tolerance of 0.171935. This typically happens when regularization is not sufficient. Consider increasing regularization.
PROGRESS: +-----------+----------+--------------+------

In [10]:
set2_poly=polynomial_sframe(set_2['sqft_living'],15)
my_features=set2_poly.column_names()
set2_poly['price']=set_2['price']
set2_0p=graphlab.linear_regression.create(set2_poly,'price',features=my_features,validation_set=None,l2_penalty=0.0)
set2_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5345
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005954     | 2108699.365891     | 241905.255747 |
PROGRESS: WARNING: Matrix is close to being singular or badly scaled. The solution is accurate only up to a tolerance of 0.638432. This typically happens when regularization is not sufficient. Consider increasing regularization.
PROGRESS: +-----------+----------+--------------+------

In [11]:
set3_poly=polynomial_sframe(set_3['sqft_living'],15)
my_features=set3_poly.column_names()
set3_poly['price']=set_3['price']
set3_0p=graphlab.linear_regression.create(set3_poly,'price',features=my_features,validation_set=None,l2_penalty=0.0)
set3_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5460
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005487     | 2040630.516510     | 245534.639353 |
PROGRESS: WARNING: Matrix is close to being singular or badly scaled. The solution is accurate only up to a tolerance of 0.0201853. This typically happens when regularization is not sufficient. Consider increasing regularization.
PROGRESS: +-----------+----------+--------------+-----

In [12]:
set4_poly=polynomial_sframe(set_4['sqft_living'],15)
my_features=set4_poly.column_names()
set4_poly['price']=set_4['price']
set4_0p=graphlab.linear_regression.create(set4_poly,'price',features=my_features,validation_set=None,l2_penalty=0.0)
set4_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5316
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005356     | 1747911.677465     | 233617.930791 |
PROGRESS: WARNING: Matrix is close to being singular or badly scaled. The solution is accurate only up to a tolerance of 1.24705. This typically happens when regularization is not sufficient. Consider increasing regularization.
PROGRESS: +-----------+----------+--------------+-------

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.) In GraphLab Create, adding an L2 penalty is a matter of adding an extra argument.

With the extra argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the extra parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [13]:
set1_poly=polynomial_sframe(set_1['sqft_living'],15)
my_features=set1_poly.column_names()
set1_poly['price']=set_1['price']
set1_0p=graphlab.linear_regression.create(set1_poly,'price',features=my_features,validation_set=None,l2_penalty=1e5)
set1_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5492
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005371     | 4434478.195911     | 359313.312739 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
+-------------+-------+-------------------+
|     name    | index |       value       |
+-------------+-------+-------------------+
| (intercept

In [14]:
set2_poly=polynomial_sframe(set_2['sqft_living'],15)
my_features=set2_poly.column_names()
set2_poly['price']=set_2['price']
set2_0p=graphlab.linear_regression.create(set2_poly,'price',features=my_features,validation_set=None,l2_penalty=1e5)
set2_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5345
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005409     | 3014798.352075     | 339161.587604 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
+-------------+-------+-------------------+
|     name    | index |       value       |
+-------------+-------+-------------------+
| (intercept

In [15]:
set3_poly=polynomial_sframe(set_3['sqft_living'],15)
my_features=set3_poly.column_names()
set3_poly['price']=set_3['price']
set3_0p=graphlab.linear_regression.create(set3_poly,'price',features=my_features,validation_set=None,l2_penalty=1e5)
set3_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5460
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005360     | 3303235.895876     | 340067.528618 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
+-------------+-------+-------------------+
|     name    | index |       value       |
+-------------+-------+-------------------+
| (intercept

In [16]:
set4_poly=polynomial_sframe(set_4['sqft_living'],15)
my_features=set4_poly.column_names()
set4_poly['price']=set_4['price']
set4_0p=graphlab.linear_regression.create(set4_poly,'price',features=my_features,validation_set=None,l2_penalty=1e5)
set4_0p.get('coefficients').print_rows(num_rows=16, num_columns=3)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 5316
PROGRESS: Number of features          : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Number of coefficients    : 16
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.005115     | 5895213.657463     | 344443.411658 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
+-------------+-------+-------------------+
|     name    | index |       value       |
+-------------+-------+-------------------+
| (intercept

These curves should vary a lot less, now that you introduced regularization.

***QUIZ QUESTION:  For the models learned with regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder:

In [17]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=0)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [18]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [19]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2770601782,2014-08-01 00:00:00+00:00,453000.0,3.0,2.5,1510.0,1618,2.5,0
8581200350,2014-06-17 00:00:00+00:00,187500.0,3.0,1.5,1180.0,7000,1,0
7524200330,2015-03-16 00:00:00+00:00,290000.0,4.0,2.0,1630.0,7618,1.5,0
3331500121,2015-02-10 00:00:00+00:00,342888.0,2.0,1.0,790.0,5150,1,0
9297300045,2014-07-09 00:00:00+00:00,550000.0,3.0,2.0,1970.0,4166,2,0
0726059344,2015-04-21 00:00:00+00:00,475000.0,3.0,2.25,1580.0,8659,1,0
3336001515,2015-05-11 00:00:00+00:00,426250.0,4.0,1.0,1610.0,6000,1.5,0
7215720070,2014-08-06 00:00:00+00:00,1250000.0,5.0,5.0,5000.0,32909,2,0
3883100220,2015-01-20 00:00:00+00:00,299000.0,3.0,1.75,2010.0,8065,1,0
7214820610,2014-10-07 00:00:00+00:00,448000.0,4.0,1.75,2560.0,8270,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [63]:
validation4=train_valid_shuffled[5818:7758]

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $549,213.

In [64]:
print int(round(validation4['price'].mean(), 0))

549213


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [30]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 2770601782 | 2014-08-01 00:00:00+00:00 | 453000.0 |   3.0    |    2.5    |
| 8581200350 | 2014-06-17 00:00:00+00:00 | 187500.0 |   3.0    |    1.5    |
| 1591600676 | 2015-03-11 00:00:00+00:00 | 246000.0 |   3.0    |    1.0    |
| 8805400010 | 2015-02-26 00:00:00+00:00 | 275500.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1510.0   |   1618   |  2.5   |     0      |  0   |     3     |   8   |    1330    |
|    1180.0   |   7000   |  

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [71]:
first_half=train_valid_shuffled[0:5818]
second_half=train_valid_shuffled[7758:19396]
train4=first_half.append(second_half)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,008.

In [72]:
print int(round(train4['price'].mean(), 0))

539008


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [74]:
def get_RSS(prediction,output):
    error=prediction-output
    RSS=(error*error).sum()
    return RSS



def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    n=len(data)
    RSS=0
    for i in range(k):
        start=(n*i)/k
        end=(n*(i+1))/k-1
        valid=data[start:end+1]
        train=(data[0:start]).append(data[end+1:n])
        model_train=graphlab.linear_regression.create(train,output_name,features=features_list,
                                                  validation_set=None,l2_penalty=l2_penalty,verbose=False)
        prediction=model_train.predict(valid)
        A=get_RSS(prediction,valid[output_name])
        RSS=RSS+A
    valid_error=RSS/k
    return valid_error
        
  

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [75]:
import numpy as np

cv_poly=polynomial_sframe(train_valid_shuffled['sqft_living'],15)
my_features=cv_poly.column_names()
cv_poly['price']=train_valid_shuffled['price']

for l2_penalty in np.logspace(1,7,num=13):
    valid_error=k_fold_cross_validation(10, l2_penalty,cv_poly,'price',my_features)
    print l2_penalty
    print valid_error
    


10.0
4.88767109398e+14
31.6227766017
2.92360235509e+14
100.0
1.6171788884e+14
316.227766017
1.21336988371e+14
1000.0
1.21561145153e+14
3162.27766017
1.23897333756e+14
10000.0
1.36512864532e+14
31622.7766017
1.72144626237e+14
100000.0
2.30352094259e+14
316227.766017
2.53935152725e+14
1000000.0
2.59301031086e+14
3162277.66017
2.63271933283e+14
10000000.0
2.65291770329e+14


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`.  This way, your final model will be trained on the entire dataset.

In [58]:
cv1_poly=polynomial_sframe(train_valid['sqft_living'],15)
my_features=cv1_poly.column_names()
cv1_poly['price']=train_valid['price']

model1 = graphlab.linear_regression.create(cv1_poly, target = 'price', features = my_features, verbose = False, 
                                           validation_set = None, l2_penalty=1000)
Val_err = k_fold_cross_validation(10, 1000, cv1_poly,'price',my_features)
print Val_err

1.14484038149e+27


***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [62]:
prediction=model1.predict(test)
RSS=get_RSS(prediction,test['price'])
print RSS

2.49143761914e+14
